In [1]:
reset()
%display latex
Partitions.options.latex="list"
load("partition_3d.sage")

`Partitions3d(lamb, mu, nu)` gives solid partitions with legs specified by $\lambda$, $\mu$, $\nu$, but since we are dealing with 0 dimensional subschemes there are no legs.

In [2]:
PPs = Partitions3d([1],[1],[1])
PPs2= Partitions3d([1],[1],[1])
V = Partition3d([(0,0,0)])
PPs.minimal_volume()

-2

In [3]:
(0, 0, 0) in V

True

The solid partitions are of form $\pi_Z=\{\pi_{ij}\}_{i,j\geq 1}$ with $\pi_{ij}\geq 0$. The corresponding ideal $I_Z$ is generated by monomials $\{x_1^{i-1}x_2^{j-1}x_3^{\pi_{ij}}\}$. The length of the ideal is $|\pi_Z|=\sum \pi_{ij}$. In the plot, the boxes are located at $(i-1,j-1,k-1)$ for $k=1,...,\pi_{ij}$, each representing an element of the basis for $\mathcal{O}_X/I_Z$.

`PPs.with_num_boxes(n)` is the set of all partitions with size $n$ (list is not ordered because python iterator).

In [4]:
PPs2 = list(PPs.with_num_boxes(10))
PPs2[0].plot(colors=('green', 'yellow', 'white'))
#easy example
easyExPt= Partition3d([(0, 0, 0)])
#complicated example 
ExPt = PPs2[0]

Graphics3d Object

In [5]:
len(PPs2)

3522

In [6]:
PPs2[0].volume()

8

`partition.character` outputs
$$Q:=\sum_{i,j\geq 1}\sum_{l=1}^{\pi_{ij}}x_1^{i-1}x_2^{j-1}x_3^{l-1}=\frac{1-P(I_Z)}{(1-x_1)(1-x_2)(1-x_3)}$$

In [7]:
wR.<x1,x2,x3,y> = LaurentPolynomialRing(ZZ)
Pt= easyExPt
# Q = Qnum / (1-x)(1-y)(1-z) 
Q = Pt.character(x1,x2,x3)
Q

1

We can the compute the virtual tangent space using `BareVertex.weight(partition)`
$$\text{tr}_{-\text{RHom}(I_Z,I_Z)_0}=Q - \frac{\bar{Q}}{x_1x_2x_3} + Q \bar{Q} \frac{(1-x_1)(1-x_2)(1-x_3)} {x_1x_2x_3}\in K_{\mathsf{T}}^*(\mathrm{pt})$$

In [8]:
load("bare_vertex.sage")
T = BareVertex.weight(Pt,x1,x2,x3)
T

x3^-1 + x2^-1 + x1^-1 - x2^-1*x3^-1 - x1^-1*x3^-1 - x1^-1*x2^-1

Mod out $x_1x_2x_3=1$. Then we can take square roots.<font color='red'>Use `descendant` function to add variable $y$?</font>

In [9]:
Tquo = BareVertex.weight(Pt,x1,x2,1/x1*1/x2)
Tquo

x1*x2 - x1 - x2 + x2^-1 + x1^-1 - x1^-1*x2^-1

`Cohomology.measure` gives the euler class $e(T_Z)\in H_{\mathsf{T}}^*(\mathrm{pt})_{\mathrm{loc}}$. Here $x_1,x_2,x_3$ are viewed as corresponding cohomology variables, as oppose to K-theory ones. After modding out the relations, we should get the euler class is $1$ by self-dual. <font color='red'>Why am I getting $(-1)^{|\pi|}$?</font>

In [10]:
eT = pretty(Cohomology.measure(T))
eT

(x1 + x2)*(x1 + x3)*(x2 + x3)/(x1*x2*x3)

In [11]:
pretty(Cohomology.measure(Tquo))

-1

Try to compute the Todd class $\text{td}(T)=e(T)/\text{ch}(\wedge^\bullet T^\vee)$. The method `KTheory.measure_unsymmetrized(w,m)` computes 
$$\frac{1}{1-1/w}=\frac{1}{1-e^{-c_1(w)}}=\frac{1}{\text{ch}(\wedge^\bullet w^\vee)}$$
<font color='red'>Here $m=1$. Can set $m=y$ for the expression in Nekrasov? Why do I get bug when setting $m=y$?</font>

In [12]:
pretty(KTheory.measure_unsymmetrized(T))

(x1*x2 - 1)*(x1*x3 - 1)*(x2*x3 - 1)/((x1 - 1)*(x2 - 1)*(x3 - 1))

This lives in $K_{\mathsf{T}}^*(\mathrm{pt})_{\mathrm{loc}}$. If we view the todd class as a power series in cohomology, we should take degree $k$ and end up in $H_{\mathsf{T}}^{k}(\mathrm{pt})_{\mathrm{loc}}$. To get degree $k$ part, we replace $x_i$ with $e^{x_i}$, convert it into Laurent polynomial, then take degree $k$. For K-theory classes we can use `Cohomology.chern_character(k, f)` which is just $c_1(-)^k/k!$. <font color='red'> In our case we have something in the field of fraction. So we need to take recipocal of a power series. Is this slow?<font>

Can modify the code for `KTheory.measure_unsymmetrized(w,m)` to compute 
$$\text{ch}(\text{det}^{\frac12}(T^\vee))=\text{ch}(K^{\frac12})$$
Now we have all the components to compute Nekrasov genus.
$$\mathcal{N}_y(\alpha^{[n]})=\sum_{|Z|=n}\text{ch}\left(\frac{\wedge^\bullet}{\text{det}^{\frac12}} \alpha^{[n]}|_Z\otimes y^{-1}\right)$$
$$I_{n}(\alpha,y)=\sum_{|Z|=n}\frac{\text{ch}((K)^{\frac12})}{\text{ch}(\wedge^\bullet T^\vee)}
\text{ch}\left(\frac{\wedge^\bullet}{\text{det}^{\frac12}} \alpha^{[n]}|_Z\otimes y^{-1}\right)$$
$$=\color{red}{(?)}\sum_{|Z|=n}\text{ch}\left(\frac{\wedge^\bullet}{\text{det}^{\frac12}} -T^\vee\oplus \alpha^{[n]}|_Z\otimes y^{-1}\right)$$

Suppose we have a rank $r$ equivariant bundle $\alpha$, with global sections $\mathbb{C}[x_1,x_2,x_3,x_4]^r$, spanned by monomials $x^j=(x_1^{j_1},...,x_4^{j_4})$ over $\mathbb{C}$. The $\mathsf{T}=\mathbb{C}^3=\{(t_1,t_2,t_3)\}$ action on $\alpha$ is determined by its action on $\mathbb{C}^r$ by equivariance. Say it has characters $\chi_i^{(j)}:t_i\mapsto t_i^{b_i^{(j)}}$ for each $j=1,...,r$ and $i=1,2,3$. Then the generating function for $\alpha^{[n]}|_Z$ is
$$\sum_{u\in\pi}\sum_{j=1}^r\prod_{i=1}^3t_i^{b_i^{(j)}+u_i}\in K_0^\mathsf{T}(\mathrm{pt})=\mathbb{Z}[t_1^{\pm1},...,t_4^{\pm1}]/(t_1...t_4=1)$$
where $u\in \pi$ means $u=(v_1-v_4,v_2-v_4,v_3-v_4)$ for some box $(v_1,...,v_4)$ in the solid partition $\pi$. The equivariant chern roots are
$$\sum_{i=1}^3(b_i^{(j)}+u_i)\lambda_i\in H_\mathsf{T}^1(\mathrm{pt})$$
for $u\in\pi$, $i=1,...,r$. 

<font color='red'>Need to write code that outputs generating function with input $b_i^{(j)}$ and $\pi$. Then we can take $c_n$ and get Verlinde number.</font> 

----------------------------------------